In [20]:
from nba_api.stats.endpoints import teaminfocommon, commonplayoffseries, teamestimatedmetrics
from nba_api.stats.static import teams
from tqdm import tqdm
import pandas as pd
import numpy as np
import plotly.express as px
import statsmodels.api as sm

In [ ]:
#Collect data

In [ ]:
team_ids = [i['id'] for i in teams.get_teams()]

In [ ]:
seasons = [
    "2010-11",
    "2011-12",
    "2012-13",
    "2013-14",
    "2014-15",
    "2015-16",
    "2016-17",
    "2017-18",
    "2018-19",
    "2019-20",
    "2021-22",
    "2022-23",
    "2023-24"
]

In [ ]:
emets = teamestimatedmetrics.TeamEstimatedMetrics(season="2015-16", season_type="Playoffs")

In [ ]:
emets = emets.get_data_frames()

In [ ]:
emets[0]

,TEAM_NAME,TEAM_ID,GP,W,L,W_PCT,MIN,E_OFF_RATING,E_DEF_RATING,E_NET_RATING,...,MIN_RANK,E_OFF_RATING_RANK,E_DEF_RATING_RANK,E_NET_RATING_RANK,E_AST_RATIO_RANK,E_OREB_PCT_RANK,E_DREB_PCT_RANK,E_REB_PCT_RANK,E_TM_TOV_PCT_RANK,E_PACE_RANK
0,Boston Celtics,1610612738,6,2,4,0.333,293.0,91.3,97.7,-6.3,...,11,14,1,10,9,12,1,8,12,1
1,Houston Rockets,1610612745,5,1,4,0.200,240.0,89.2,110.1,-20.9,...,13,16,13,15,14,1,7,4,16,1
2,Golden State Warriors,1610612744,24,15,9,0.625,1157.0,108.4,102.1,6.3,...,1,3,7,3,2,7,12,10,10,3
3,Portland Trail Blazers,1610612757,11,5,6,0.455,533.0,104.5,108.7,-4.2,...,6,6,12,9,7,5,9,5,6,4
4,Atlanta Hawks,1610612737,10,4,6,0.400,485.0,100.8,101.6,-0.8,...,7,10,5,8,3,13,11,9,13,5
5,Oklahoma City Thunder,1610612760,18,11,7,0.611,864.0,108.5,103.2,5.3,...,4,2,8,4,8,2,4,1,11,6
6,LA Clippers,1610612746,6,2,4,0.333,288.0,102.2,101.8,0.4,...,12,9,6,7,11,8,14,12,1,7
7,Dallas Mavericks,1610612742,5,1,4,0.200,240.0,99.3,117.7,-18.4,...,13,11,16,14,6,11,16,16,5,8
8,San Antonio Spurs,1610612759,10,6,4,0.600,480.0,107.1,99.5,7.6,...,8,5,4,2,4,14,13,13,4,9
9,Charlotte Hornets,1610612766,7,3,4,0.429,336.0,96.4,106.5,-10.1,...,9,13,11,13,16,16,8,15,2,10


In [ ]:
reg_season_data = []
playoff_data = []
for season in tqdm(seasons):
  reg_season = teamestimatedmetrics.TeamEstimatedMetrics(season=season, season_type="Regular Season")
  reg_season = reg_season.get_data_frames()[0]
  reg_season["SEASON"] = [season] * len(reg_season)

  playoffs = teamestimatedmetrics.TeamEstimatedMetrics(season=season, season_type="Playoffs")
  playoffs = playoffs.get_data_frames()[0]
  playoffs["SEASON"] = [season] * len(playoffs)

  reg_season_data.append(reg_season)
  playoff_data.append(playoffs)

reg_season_data = pd.concat(reg_season_data)
playoff_data = pd.concat(playoff_data)

reg_season_data.to_csv("reg_season_data.csv")
playoff_data.to_csv("playoff_data.csv")

100%|██████████| 13/13 [00:19<00:00,  1.52s/it]


In [ ]:
reg_season_data

,TEAM_NAME,TEAM_ID,GP,W,L,W_PCT,MIN,E_OFF_RATING,E_DEF_RATING,E_NET_RATING,...,E_OFF_RATING_RANK,E_DEF_RATING_RANK,E_NET_RATING_RANK,E_AST_RATIO_RANK,E_OREB_PCT_RANK,E_DREB_PCT_RANK,E_REB_PCT_RANK,E_TM_TOV_PCT_RANK,E_PACE_RANK,SEASON
0,Minnesota Timberwolves,1610612750,82,17,65,0.207,3956.0,101.1,108.3,-7.2,...,24,27,28,29,2,16,7,30,1,2010-11
1,New York Knicks,1610612752,82,42,40,0.512,3956.0,108.3,106.9,1.4,...,5,21,13,20,24,26,28,4,2,2010-11
2,Denver Nuggets,1610612743,82,50,32,0.610,3941.0,109.5,104.8,4.7,...,1,16,8,12,27,5,13,9,3,2010-11
3,Sacramento Kings,1610612758,82,24,58,0.293,3971.0,100.6,106.3,-5.7,...,26,20,25,26,1,13,4,28,4,2010-11
4,Golden State Warriors,1610612744,82,36,46,0.439,3966.0,105.5,107.6,-2.1,...,13,26,20,11,14,30,30,16,5,2010-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,Minnesota Timberwolves,1610612750,55,39,16,0.709,2655.0,113.5,105.9,7.6,...,16,1,3,14,20,3,3,28,26,2023-24
26,Miami Heat,1610612748,55,30,25,0.545,2645.0,111.5,110.8,0.7,...,22,8,16,16,25,2,20,14,27,2023-24
27,New York Knicks,1610612752,55,33,22,0.600,2640.0,114.7,111.7,3.0,...,12,11,10,29,1,7,1,16,28,2023-24
28,Denver Nuggets,1610612743,55,36,19,0.655,2640.0,114.6,112.4,2.2,...,13,14,11,2,5,17,10,9,29,2023-24


In [13]:
playoff_data

,TEAM_NAME,TEAM_ID,GP,W,L,W_PCT,MIN,E_OFF_RATING,E_DEF_RATING,E_NET_RATING,...,E_OFF_RATING_RANK,E_DEF_RATING_RANK,E_NET_RATING_RANK,E_AST_RATIO_RANK,E_OREB_PCT_RANK,E_DREB_PCT_RANK,E_REB_PCT_RANK,E_TM_TOV_PCT_RANK,E_PACE_RANK,SEASON
0,Denver Nuggets,1610612743,5,1,4,0.200,240.0,99.8,106.3,-6.6,...,11,13,13,11,16,11,16,2,1,2010-11
1,New York Knicks,1610612752,4,0,4,0.000,192.0,95.9,104.2,-8.3,...,16,11,14,14,4,13,11,14,2,2010-11
2,Indiana Pacers,1610612754,5,1,4,0.200,240.0,96.8,103.3,-6.5,...,15,7,12,15,5,16,14,15,3,2010-11
3,Oklahoma City Thunder,1610612760,17,9,8,0.529,841.0,106.9,103.3,3.6,...,2,8,2,13,2,6,2,13,4,2010-11
4,Memphis Grizzlies,1610612763,13,7,6,0.538,649.0,101.7,104.0,-2.3,...,7,10,8,9,7,7,6,3,5,2010-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,Minnesota Timberwolves,1610612750,5,1,4,0.200,245.0,107.9,115.1,-7.2,...,11,12,13,10,15,8,14,9,12,2022-23
12,New York Knicks,1610612752,11,6,5,0.545,528.0,105.4,105.1,0.3,...,13,2,4,16,1,6,2,15,13,2022-23
13,Cleveland Cavaliers,1610612739,5,1,4,0.200,240.0,101.1,104.6,-3.6,...,14,1,9,9,9,16,15,16,14,2022-23
14,Philadelphia 76ers,1610612755,11,7,4,0.636,533.0,108.2,110.2,-2.0,...,10,9,8,14,8,2,3,8,15,2022-23


In [14]:
percent_change_off_rating = []
percent_change_def_rating = []
percent_change_net_rating = []
playoff_win_pct = []
playoff_games_played = []

In [15]:
for index, row in playoff_data.iterrows():
  team_reg_season = reg_season_data[reg_season_data["TEAM_ID"] == row["TEAM_ID"]]
  team_reg_season = team_reg_season[team_reg_season["SEASON"] == row["SEASON"]]

  if len(team_reg_season) > 0:
    percent_change_off_rating.append((row["E_OFF_RATING"] - team_reg_season["E_OFF_RATING"].values[0]) / team_reg_season["E_OFF_RATING"].values[0])
    percent_change_def_rating.append((row["E_DEF_RATING"] - team_reg_season["E_DEF_RATING"].values[0]) / team_reg_season["E_DEF_RATING"].values[0])
    percent_change_net_rating.append((row["E_NET_RATING"] - team_reg_season["E_NET_RATING"].values[0]) / team_reg_season["E_NET_RATING"].values[0])
    playoff_win_pct.append(row["W_PCT"])
    playoff_games_played.append(row["GP"])

<ipython-input-15-0b8b5dd47766>:8: RuntimeWarning: divide by zero encountered in scalar divide
  percent_change_net_rating.append((row["E_NET_RATING"] - team_reg_season["E_NET_RATING"].values[0]) / team_reg_season["E_NET_RATING"].values[0])


In [16]:
training_data = {
    "percent_change_off_rating": percent_change_off_rating,
    "percent_change_def_rating": percent_change_def_rating,
    "percent_change_net_rating": percent_change_net_rating,
    "playoff_win_pct": playoff_win_pct,
    "playoff_games_played": playoff_games_played
}

training_data = pd.DataFrame(training_data)

In [18]:
training_data.dropna(inplace=True)

In [23]:
#Percent change in offensive rating and playoff win percent
X = training_data["percent_change_off_rating"]
y = training_data["playoff_win_pct"]

model = sm.OLS(y, sm.add_constant(X)).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:        playoff_win_pct   R-squared:                       0.199
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     47.22
Date:                Tue, 20 Feb 2024   Prob (F-statistic):           8.82e-11
Time:                        19:16:44   Log-Likelihood:                 51.102
No. Observations:                 192   AIC:                            -98.20
Df Residuals:                     190   BIC:                            -91.69
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [30]:
#Percent change in offensive rating and playoff games played
X = training_data["percent_change_off_rating"]
y = training_data["playoff_games_played"]

model = sm.OLS(y, sm.add_constant(X)).fit()

print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     playoff_games_played   R-squared:                       0.173
Model:                              OLS   Adj. R-squared:                  0.168
Method:                   Least Squares   F-statistic:                     39.65
Date:                  Tue, 20 Feb 2024   Prob (F-statistic):           2.05e-09
Time:                          19:19:23   Log-Likelihood:                -596.57
No. Observations:                   192   AIC:                             1197.
Df Residuals:                       190   BIC:                             1204.
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
co

In [28]:
#Percent change in defensive rating and playoff win percentage
X = training_data["percent_change_def_rating"]
y = training_data["playoff_win_pct"]

model = sm.OLS(y, sm.add_constant(X)).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        playoff_win_pct   R-squared:                       0.210
Model:                            OLS   Adj. R-squared:                  0.206
Method:                 Least Squares   F-statistic:                     50.56
Date:                Tue, 20 Feb 2024   Prob (F-statistic):           2.28e-11
Time:                        19:18:17   Log-Likelihood:                 52.444
No. Observations:                 192   AIC:                            -100.9
Df Residuals:                     190   BIC:                            -94.37
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [37]:
#Percent change in defensive rating and playoff games played
X = training_data["percent_change_def_rating"]
y = training_data["playoff_games_played"]

model = sm.OLS(y, sm.add_constant(X)).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     playoff_games_played   R-squared:                       0.101
Model:                              OLS   Adj. R-squared:                  0.096
Method:                   Least Squares   F-statistic:                     21.36
Date:                  Tue, 20 Feb 2024   Prob (F-statistic):           7.03e-06
Time:                          19:21:46   Log-Likelihood:                -604.54
No. Observations:                   192   AIC:                             1213.
Df Residuals:                       190   BIC:                             1220.
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
co

In [34]:
training_data

,percent_change_off_rating,percent_change_def_rating,percent_change_net_rating,playoff_win_pct,playoff_games_played
0,-0.088584,0.014313,-2.404255,0.200,5
1,-0.114497,-0.025257,-6.928571,0.000,4
2,-0.050049,-0.000967,3.333333,0.200,5
3,-0.015654,-0.006731,-0.200000,0.529,17
4,-0.025862,0.014634,-2.210526,0.538,13
...,...,...,...,...,...
187,-0.030548,0.036004,-37.000000,0.200,5
188,-0.082681,-0.059928,-0.903226,0.545,11
189,-0.110818,-0.028784,-1.600000,0.200,5
190,-0.063203,0.000000,-1.384615,0.636,11


In [38]:
#Percent change in net rating and playoff win percent
X = training_data["percent_change_net_rating"]
y = training_data["playoff_games_played"]

model = sm.OLS(y, sm.add_constant(X)).fit()

print(model.summary())

LinAlgError: SVD did not converge

In [ ]:
#Visualizations